In [88]:
#re.findall('\(.*?\)',b1)
re.findall('\(.*?\)',f1)

['(2004)']

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [287]:
wikipages = ['https://en.wikipedia.org/wiki/List_of_children%27s_books_made_into_feature_films?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(0%E2%80%939,_A%E2%80%93C)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(D%E2%80%93J)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(K%E2%80%93R)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(S%E2%80%93Z)?oldformat=true'
            ]

In [288]:
def add_to_rows(page_soup):
    global rows
    
    tables = page_soup.find_all('table', {'class': 'wikitable'})

    for alphabet in tables:
        rows += alphabet.findAll('tr')
    
    print(len(tables), "and", len(rows))

In [291]:
def add_to_book_film_dict():
    for row in rows:
        cols = row.findAll('td')

        try:
            book_title_year = cols[0].text.replace('\n', '').split(',')[0]
            author = cols[0].text.replace('\n', '').split(',')[-1].lstrip() 
    
            adaptations = cols[1].text.replace('\n', '').rstrip()
            adaptations = list(adaptations.split(')'))
    
            for index, title in enumerate(adaptations):
                if ('TV' in title):
                    adaptations[index - 1] += ' (TV)'
                    del adaptations[index]
            
                if len(title) > 0:
                    adaptations[index] += ')'
        
            adaptations = adaptations[:len(adaptations) - 1]

                 
            book_film_dict[book_title_year] = {'author': author, 
                                               'count': len(adaptations), 
                                               'adaptations': adaptations}
        except IndexError:
            pass
        
    print(len(book_film_dict))

In [290]:
rows = []
book_film_dict = {}

for url in wikipages:
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'lxml')
    
    add_to_rows(soup)
    add_to_book_film_dict()

25 and 376
348
3 and 798
732
7 and 1245
1143
8 and 1662
1513
7 and 2034
1845
